<a href="https://colab.research.google.com/github/xiaotiansu/writing-assistance/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 18.3 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171047 sha256=1491e3d0bad24b44266ee1a0efe8f0d87a67a5bf2b7be6c43fcd67342759be26
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
pip install flair

# Cleaning

In [4]:
import re
import random
import string
import pandas as pd
from cleantext import clean
from bs4 import BeautifulSoup
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-german-large")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:591: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-11-15 11:42:09,971 loading file /root/.flair/models/ner-german-large/6b8de9edd73722050be2547acf64c037b2df833c6e8f0e88934de08385e26c1e.4b0797effcc6ebb1889d5d29784b97f0a099c1569b319d87d7c387e44e2bba48


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

2022-11-15 11:42:44,278 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, B-PER, E-PER, S-LOC, B-MISC, I-MISC, E-MISC, S-PER, B-ORG, E-ORG, S-ORG, I-ORG, B-LOC, E-LOC, S-MISC, I-PER, I-LOC, <START>, <STOP>


In [19]:
df = pd.read_csv('/content/drive/MyDrive/data/reviews_html18-21.csv', header=0).reset_index(drop=True)
df['cleaned'] = df['review']
df

,year,course,review,cleaned
0,2018,sg_bi2,<p>Ich finde die Personas sehr gut. Vlt ist di...,<p>Ich finde die Personas sehr gut. Vlt ist di...
1,2018,sg_bi2,<p>Lieber Kommilitone oder Kommilitonin 646</p...,<p>Lieber Kommilitone oder Kommilitonin 646</p...
2,2018,sg_bi2,<p>1.) Worin sehen Sie die Stärken der Lösung ...,<p>1.) Worin sehen Sie die Stärken der Lösung ...
3,2018,sg_bi2,<p>Du hast die Modellierung sehr gut und nachv...,<p>Du hast die Modellierung sehr gut und nachv...
4,2018,sg_bi2,<p>Stärken:</p><p>- Spannendes Geschäftsm...,<p>Stärken:</p><p>- Spannendes Geschäftsm...
...,...,...,...,...
8829,2021,ka_bwl1a,<p>&gt;</p><p>Ich würde dir empfehlen zwischen...,<p>&gt;</p><p>Ich würde dir empfehlen zwischen...
8830,2021,ka_bwl1a,<p>1.) Worin sehen Sie die Stärken der Lösung ...,<p>1.) Worin sehen Sie die Stärken der Lösung ...
8831,2021,ka_bwl1a,<p>Review</p><p><br></p><p>Die Bearbeitung der...,<p>Review</p><p><br></p><p>Die Bearbeitung der...
8832,2021,ka_bwl1a,"<p>Hallo,</p><p><br></p><p>deine Ausarbeitung ...","<p>Hallo,</p><p><br></p><p>deine Ausarbeitung ..."


In [14]:
def clean_text(review):
    # remove html tags
    sentence = BeautifulSoup(review).get_text(" ")
    # replace indexing with a space
    sentence = re.sub(r"\d+[).]+", ' ', sentence)
    # remove .pdf files
    sentence = re.sub('\w*.pdf', '', sentence)
    # keep e3 values
    sentence = re.sub('[eE]3', 'E-three', sentence)
    # remove version numbers
    sentence = re.sub('[vV][12]', '', sentence)
    # replace new line with space
    sentence = sentence.replace('\n', ' ')
    # remove urls
    sentence = re.sub(r'http\S+', '', sentence)

    # use package
    sentence = clean(sentence, no_emoji=True, lower=False,
        no_urls=True, no_emails=True, no_phone_numbers=True, no_numbers=True,
        no_digits=True, no_currency_symbols=True, no_punct=False,
        replace_with_url='', replace_with_email='', replace_with_phone_number='',
        replace_with_number='', replace_with_digit='', replace_with_currency_symbol='', lang='de')
    return sentence

In [15]:
def expand_abbreviations(review):
    abbr = {
      **dict.fromkeys(['ggf.', 'ggf'], 'gegebenenfalls'),
      **dict.fromkeys(['vlt.', 'vlt'], 'vielleicht'),
      **dict.fromkeys(['dh.', 'd.h.', 'd.h', 'd h', 'dh'], 'da her'),
      **dict.fromkeys(['zb.', 'z.b.', 'z.b', 'z b', 'zb'], 'zum Beispiel'),
      **dict.fromkeys(['bsp.', 'bsp', 'bspw.', 'bspw'], 'beispielsweise'),
      **dict.fromkeys(['oä.', 'o.ä.', 'o.ä', 'oä'], 'oder ähnliches'),
      **dict.fromkeys(['ev.', 'ev', 'evtl.', 'evtl'], 'eventuell'),
      '&amp': 'und',
    }
    for word, expansion in abbr.items():
      # case insensitive
      compiled = re.compile(re.escape(word), re.IGNORECASE)
      review = compiled.sub(expansion, review)
    return review

In [8]:
def remove_keywords(review):
    # anonymize_remove
    keywords = keywords = ['Business Engineering', 'Business Innovation',
                           'Rietsche', 'Leimeister', 'Söllner', 'Peters',
                           'HSG', 'Kassel', 'St. Gallen', 'St Gallen', 'Gallen']
    for k in keywords:
      # case insensitive
      compiled = re.compile(re.escape(k), re.IGNORECASE)
      review = compiled.sub('', review)
    return review

In [9]:
def clean_review(review):
    # remove meaningless characters
    review = clean_text(review)

    review = expand_abbreviations(review)

    review = remove_keywords(review)

    # remove template questions
    questions = ["Worin sehen Sie die Stärken der Lösung des Kommilitonen?",
                 "Worin sehen Sie die Stärken der Lösung des Kommilitonen ",
                "Worin sehen Sie Schwächen der Lösung des Kommilitonen und wie können diese behoben werden?",
                "Worin sehen Sie Schwächen der Lösung des Kommilitonen und wie können diese behoben werden ",
                "Worauf sollte in der Überarbeitung der Lösung geachtet werden?",
                "Worauf sollte in der Überarbeitung der Lösung geachtet werden ",
                "Geben Sie hierbei konkrete Verbesserungsvorschläge",
                "Geben Sie hierbei konkrete Verbesserungsvorschläge (konstruktives Feedback)"]

    for q in questions:
      review = review.replace(q, '')
    # remove all characters before the first letter
    return review.lstrip(string.punctuation + string.whitespace)

In [20]:
total_ners = 0

for i in range(0, len(df)):
    if i % 500 == 0:
        print(i, ': ', total_ners)
    review = clean_review(df['review'][i])
    sentence = Sentence(review)
    tagger.predict(sentence)
    ners = sentence.get_spans('ner')
    total_ners += len(ners)
    if ners:
      for ner in reversed(ners):
          if ner.tag == 'PER':
              replace = 'Person'
          if ner.tag == 'LOC':
              replace = 'Ort'
          if ner.tag == 'ORG':
              replace = 'Firma'
          if ner.tag == 'MISC':
              replace = '' 
          review = review[:ner.start_position] + replace + review[ner.end_position:]
    df['cleaned'][i] = review


0 :  0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


500 :  1401
1000 :  2649
1500 :  3832
2022-11-15 11:56:24,469 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-11-15 11:56:24,500 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2000 :  5026
2022-11-15 11:57:38,980 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-11-15 11:57:38,983 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-11-15 11:57:38,987 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2500 :  6109
2022-11-15 11:58:51,618 Warning: An empty Sentence was created! Are there empty strings in your dataset?
3000 :  7428
3500 :  8920
4000 :  10319
4500 :  11408
5000 :  12110
2022-11-15 12:04:33,635 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-11-15 12:05:00,052 Warning: An empty Sentence was created! Are there empty strings in your dataset?
5500 :  12629
2022-11-15 12:

In [21]:
df

,year,course,review,cleaned
0,2018,sg_bi2,<p>Ich finde die Personas sehr gut. Vlt ist di...,Ich finde die Personas sehr gut. vielleicht is...
1,2018,sg_bi2,<p>Lieber Kommilitone oder Kommilitonin 646</p...,Lieber Kommilitone oder Kommilitonin Ich habe ...
2,2018,sg_bi2,<p>1.) Worin sehen Sie die Stärken der Lösung ...,"Ausarbeitung beider Perspektiven, Kunde und Pr..."
3,2018,sg_bi2,<p>Du hast die Modellierung sehr gut und nachv...,Du hast die Modellierung sehr gut und nachvoll...
4,2018,sg_bi2,<p>Stärken:</p><p>- Spannendes Geschäftsm...,Stärken: -Spannendes Geschäftsmodell. Gute Wah...
...,...,...,...,...
8829,2021,ka_bwl1a,<p>&gt;</p><p>Ich würde dir empfehlen zwischen...,Ich würde dir empfehlen zwischen Kernnmarkt un...
8830,2021,ka_bwl1a,<p>1.) Worin sehen Sie die Stärken der Lösung ...,"Eine Stärke, die mir aufgefallen ist, dass du ..."
8831,2021,ka_bwl1a,<p>Review</p><p><br></p><p>Die Bearbeitung der...,Person Die Bearbeitung der Aufgabe ist dir ext...
8832,2021,ka_bwl1a,"<p>Hallo,</p><p><br></p><p>deine Ausarbeitung ...","Hallo, deine Ausarbeitung ist echt sehr detail..."


In [22]:
df.to_csv('/content/drive/MyDrive/data/cleaned_reviews_html18-21.csv')

# Concatenate reviews and split data

In [26]:
data_dir = '/content/drive/MyDrive/data/'
df16 = pd.read_csv(data_dir + 'cleaned_reviews_html16.csv').reset_index(drop=True)
df17 = pd.read_csv(data_dir + 'cleaned_reviews_html17.csv').reset_index(drop=True)
df18 = pd.read_csv(data_dir + 'cleaned_reviews_html18-21.csv').reset_index(drop=True)

In [37]:
empty = []
for i in range(len(df18['cleaned'])):
    if type(df18['cleaned'][i]) == float:
    # if df18['cleaned'][i] == '':
        print(i, df18['cleaned'][i])
        empty.append(i)
empty

1666 nan
1668 nan
2140 nan
2141 nan
2142 nan
2644 nan
5082 nan
5440 nan
5563 nan
5799 nan


[1666, 1668, 2140, 2141, 2142, 2644, 5082, 5440, 5563, 5799]

In [ ]:
df18.iloc[1666]

year        2019
course    sg_bi2
review       NaN
Name: 1666, dtype: object

In [38]:
new18 = df18.drop(empty).reset_index(drop=True)

In [39]:
for i in range(len(new18['cleaned'])):
    if type(new18['cleaned'][i]) == float:
        print(i, new18['cleaned'][i])

In [44]:
all_reviews = list(df16['cleaned']) + list(df17['cleaned']) + list(new18['cleaned'])
total = len(all_reviews)

total

11915

In [45]:
random.shuffle(all_reviews)
train_reviews = all_reviews[:int(total * 0.8)]
test_reviews = all_reviews[int(total * 0.8): ]
# val_reviews = all_reviews[int(total * 0.9):]

train_text = '\n\n'.join(train_reviews)
test_text = '\n\n'.join(test_reviews)
# val_text = '\n\n'.join(val_reviews)

In [46]:
with open(data_dir + 'train_reviews.txt', 'w') as f:
    f.write(train_text)
with open(data_dir + 'test_reviews.txt', 'w') as f:
    f.write(test_text)
# with open(data_dir + 'val_reviews.txt', 'w') as f:
#     f.write(val_text)